## Imports

In [24]:
# pip install --upgrade scikit-learn

In [174]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
from sklearn.preprocessing import OrdinalEncoder
from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.3.2
Sklearn  1.0


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [175]:
DATA_PATH = "../10. Robust ML/"

## Load data

In [176]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [177]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [178]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [179]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = None
### BEGIN SOLUTION
get_Title_from_Name = lambda x: x.split(',')[1].split('.')[0].strip()
df["Title"] = df["Name"].map(get_Title_from_Name)
df_test["Title"] = df_test["Name"].map(get_Title_from_Name)
### END SOLUTION



In [180]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [181]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [182]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =
### BEGIN SOLUTION
df['Title'] = df["Title"].map(title_dictionary)
df_test['Title'] = df_test["Title"].map(title_dictionary)
### END SOLUTION

In [183]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [184]:
df["Ticket"]

PassengerId
1             A/5 21171
2              PC 17599
3      STON/O2. 3101282
4                113803
5                373450
             ...       
887              211536
888              112053
889          W./C. 6607
890              111369
891              370376
Name: Ticket, Length: 891, dtype: object

In [185]:
df_test["Ticket"]

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [186]:
df["Cabin"]

PassengerId
1       NaN
2       C85
3       NaN
4      C123
5       NaN
       ... 
887     NaN
888     B42
889     NaN
890    C148
891     NaN
Name: Cabin, Length: 891, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [208]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [188]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [189]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

### BEGIN SOLUTION
# Drop other vars not specified in num_vars or cat_vars
cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(fill_value='missing', strategy='constant')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=10))])
                                    

num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='mean'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
    ], remainder='drop')
### END SOLUTION

In [190]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [191]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [192]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":
  "Random Forest":
  "AdaBoost":
  "Skl GBM":
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
"""

### BEGIN SOLUTION
tree_classifiers = {
    "Decision Tree": DecisionTreeClassifier(),
    "Extra Trees": ExtraTreesClassifier(n_estimators=1000, max_depth=9, random_state=10, min_samples_leaf=4),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Skl GBM": GradientBoostingClassifier(),
    "SKl HistGBM": HistGradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier()   
}
    
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

### END SOLUTIONv



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=10))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [193]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [202]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
### BEGIN SOLUTION
# Recommended for reproducibility
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)
### END SOLUTION



"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

### BEGIN SOLUTION

results = []
for name, model in tree_classifiers.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    prediction = model.predict(x_test)
    
    results.append({
        'ModelName': name,
        'Accuracy': (prediction == y_test).mean()*100,
        'Bal Acc.': metrics.balanced_accuracy_score(y_test, prediction)*100,
        'Time': total_time})
    
results_ord = pd.DataFrame(results)

### END SOLUTION



C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:57:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6896726	total: 889us	remaining: 889ms
1:	learn: 0.6840036	total: 2.62ms	remaining: 1.31s
2:	learn: 0.6784911	total: 4.3ms	remaining: 1.43s
3:	learn: 0.6726585	total: 5.99ms	remaining: 1.49s
4:	learn: 0.6659867	total: 7.69ms	remaining: 1.53s
5:	learn: 0.6602370	total: 9.35ms	remaining: 1.55s
6:	learn: 0.6544571	total: 11.1ms	remaining: 1.57s
7:	learn: 0.6500395	total: 12.1ms	remaining: 1.5s
8:	learn: 0.6447847	total: 13.8ms	remaining: 1.52s
9:	learn: 0.6418760	total: 14.9ms	remaining: 1.48s
10:	learn: 0.6367787	total: 16.5ms	remaining: 1.48s
11:	learn: 0.6315598	total: 18ms	remaining: 1.48s
12:	learn: 0.6262804	total: 19.6ms	remaining: 1.49s

244:	learn: 0.3566622	total: 332ms	remaining: 1.02s
245:	learn: 0.3563771	total: 334ms	remaining: 1.02s
246:	learn: 0.3562891	total: 335ms	remaining: 1.02s
247:	learn: 0.3562309	total: 336ms	remaining: 1.02s
248:	learn: 0.3562073	total: 337ms	remaining: 1.02s
249:	learn: 0.3558459	total: 339ms	remaining: 1.01s
250:	learn: 0.3558232	total: 340ms	remaining: 1.01s
251:	learn: 0.3554627	total: 341ms	remaining: 1.01s
252:	learn: 0.3551717	total: 343ms	remaining: 1.01s
253:	learn: 0.3550173	total: 345ms	remaining: 1.01s
254:	learn: 0.3545542	total: 346ms	remaining: 1.01s
255:	learn: 0.3543357	total: 348ms	remaining: 1.01s
256:	learn: 0.3541371	total: 350ms	remaining: 1.01s
257:	learn: 0.3540332	total: 352ms	remaining: 1.01s
258:	learn: 0.3537374	total: 354ms	remaining: 1.01s
259:	learn: 0.3533754	total: 355ms	remaining: 1.01s
260:	learn: 0.3530529	total: 358ms	remaining: 1.01s
261:	learn: 0.3528136	total: 359ms	remaining: 1.01s
262:	learn: 0.3523531	total: 361ms	remaining: 1.01s
263:	learn: 

492:	learn: 0.3182301	total: 674ms	remaining: 693ms
493:	learn: 0.3181408	total: 675ms	remaining: 692ms
494:	learn: 0.3179903	total: 677ms	remaining: 690ms
495:	learn: 0.3179239	total: 678ms	remaining: 689ms
496:	learn: 0.3178107	total: 680ms	remaining: 688ms
497:	learn: 0.3177950	total: 681ms	remaining: 686ms
498:	learn: 0.3176750	total: 682ms	remaining: 685ms
499:	learn: 0.3174416	total: 684ms	remaining: 684ms
500:	learn: 0.3173492	total: 685ms	remaining: 682ms
501:	learn: 0.3171966	total: 686ms	remaining: 681ms
502:	learn: 0.3171106	total: 688ms	remaining: 680ms
503:	learn: 0.3170113	total: 689ms	remaining: 678ms
504:	learn: 0.3167739	total: 691ms	remaining: 677ms
505:	learn: 0.3166181	total: 693ms	remaining: 677ms
506:	learn: 0.3164400	total: 695ms	remaining: 676ms
507:	learn: 0.3163584	total: 696ms	remaining: 674ms
508:	learn: 0.3161467	total: 697ms	remaining: 673ms
509:	learn: 0.3160345	total: 699ms	remaining: 671ms
510:	learn: 0.3159756	total: 700ms	remaining: 670ms
511:	learn: 

738:	learn: 0.2928238	total: 1.02s	remaining: 359ms
739:	learn: 0.2928168	total: 1.02s	remaining: 357ms
740:	learn: 0.2926645	total: 1.02s	remaining: 356ms
741:	learn: 0.2924042	total: 1.02s	remaining: 355ms
742:	learn: 0.2923542	total: 1.02s	remaining: 353ms
743:	learn: 0.2922378	total: 1.02s	remaining: 352ms
744:	learn: 0.2922085	total: 1.02s	remaining: 351ms
745:	learn: 0.2921777	total: 1.03s	remaining: 350ms
746:	learn: 0.2921278	total: 1.03s	remaining: 348ms
747:	learn: 0.2920218	total: 1.03s	remaining: 347ms
748:	learn: 0.2918347	total: 1.03s	remaining: 346ms
749:	learn: 0.2916605	total: 1.03s	remaining: 345ms
750:	learn: 0.2915611	total: 1.04s	remaining: 344ms
751:	learn: 0.2914606	total: 1.04s	remaining: 343ms
752:	learn: 0.2913412	total: 1.04s	remaining: 341ms
753:	learn: 0.2913059	total: 1.04s	remaining: 340ms
754:	learn: 0.2912210	total: 1.04s	remaining: 339ms
755:	learn: 0.2911274	total: 1.04s	remaining: 337ms
756:	learn: 0.2908017	total: 1.05s	remaining: 336ms
757:	learn: 

982:	learn: 0.2685788	total: 1.36s	remaining: 23.5ms
983:	learn: 0.2682260	total: 1.36s	remaining: 22.2ms
984:	learn: 0.2680357	total: 1.36s	remaining: 20.8ms
985:	learn: 0.2679318	total: 1.36s	remaining: 19.4ms
986:	learn: 0.2678159	total: 1.37s	remaining: 18ms
987:	learn: 0.2677220	total: 1.37s	remaining: 16.6ms
988:	learn: 0.2676569	total: 1.37s	remaining: 15.2ms
989:	learn: 0.2676259	total: 1.37s	remaining: 13.9ms
990:	learn: 0.2674319	total: 1.37s	remaining: 12.5ms
991:	learn: 0.2673158	total: 1.37s	remaining: 11.1ms
992:	learn: 0.2672626	total: 1.38s	remaining: 9.7ms
993:	learn: 0.2671601	total: 1.38s	remaining: 8.31ms
994:	learn: 0.2670508	total: 1.38s	remaining: 6.93ms
995:	learn: 0.2667413	total: 1.38s	remaining: 5.55ms
996:	learn: 0.2666946	total: 1.38s	remaining: 4.16ms
997:	learn: 0.2665183	total: 1.38s	remaining: 2.77ms
998:	learn: 0.2664812	total: 1.39s	remaining: 1.39ms
999:	learn: 0.2662116	total: 1.39s	remaining: 0us


In [203]:
results_ord

,ModelName,Accuracy,Bal Acc.,Time
0,Decision Tree,77.653631,75.876153,0.015957
1,Extra Trees,80.446927,77.878788,0.817809
2,Random Forest,78.770950,76.245059,0.140625
3,AdaBoost,79.888268,77.964427,0.078789
4,Skl GBM,84.916201,82.595520,0.087765
5,SKl HistGBM,81.005587,78.603426,0.638292
6,XGBoost,81.564246,78.787879,0.247339
7,LightGBM,81.005587,78.333333,0.090741
8,CatBoost,79.329609,76.969697,1.877452


In [204]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [216]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
### BEGIN SOLUTION
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
### END SOLUTION

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

### BEGIN SOLUTION

results = []
for name, model in tree_classifiers.items():
    start_time = time.time()
    prediction = model_selection.cross_val_predict(model, x, y, cv=skf)
    total_time = time.time() - start_time
    
    results.append({
        'ModelName': name,
        'Accuracy': metrics.accuracy_score(y, prediction)*100,
        'Bal Acc.': metrics.balanced_accuracy_score(y, prediction)*100,
        'Time': total_time})
    
results_ord = pd.DataFrame(results)

### END SOLUTION


C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:07:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:07:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

115:	learn: 0.4159721	total: 167ms	remaining: 1.27s
116:	learn: 0.4151439	total: 168ms	remaining: 1.27s
117:	learn: 0.4143578	total: 170ms	remaining: 1.27s
118:	learn: 0.4133795	total: 172ms	remaining: 1.27s
119:	learn: 0.4123526	total: 174ms	remaining: 1.27s
120:	learn: 0.4115075	total: 176ms	remaining: 1.28s
121:	learn: 0.4109515	total: 177ms	remaining: 1.27s
122:	learn: 0.4099755	total: 179ms	remaining: 1.28s
123:	learn: 0.4090155	total: 181ms	remaining: 1.28s
124:	learn: 0.4082462	total: 183ms	remaining: 1.28s
125:	learn: 0.4079632	total: 184ms	remaining: 1.28s
126:	learn: 0.4076868	total: 186ms	remaining: 1.27s
127:	learn: 0.4075658	total: 186ms	remaining: 1.27s
128:	learn: 0.4071054	total: 188ms	remaining: 1.27s
129:	learn: 0.4065727	total: 190ms	remaining: 1.27s
130:	learn: 0.4061682	total: 192ms	remaining: 1.27s
131:	learn: 0.4054516	total: 194ms	remaining: 1.27s
132:	learn: 0.4048088	total: 196ms	remaining: 1.28s
133:	learn: 0.4039057	total: 198ms	remaining: 1.28s
134:	learn: 

304:	learn: 0.3508882	total: 509ms	remaining: 1.16s
305:	learn: 0.3507900	total: 511ms	remaining: 1.16s
306:	learn: 0.3505926	total: 513ms	remaining: 1.16s
307:	learn: 0.3503082	total: 515ms	remaining: 1.16s
308:	learn: 0.3500987	total: 517ms	remaining: 1.16s
309:	learn: 0.3499827	total: 519ms	remaining: 1.15s
310:	learn: 0.3498993	total: 520ms	remaining: 1.15s
311:	learn: 0.3496183	total: 522ms	remaining: 1.15s
312:	learn: 0.3494765	total: 524ms	remaining: 1.15s
313:	learn: 0.3492717	total: 526ms	remaining: 1.15s
314:	learn: 0.3491778	total: 528ms	remaining: 1.15s
315:	learn: 0.3490749	total: 529ms	remaining: 1.15s
316:	learn: 0.3490311	total: 531ms	remaining: 1.14s
317:	learn: 0.3489781	total: 532ms	remaining: 1.14s
318:	learn: 0.3489236	total: 534ms	remaining: 1.14s
319:	learn: 0.3488378	total: 535ms	remaining: 1.14s
320:	learn: 0.3487540	total: 537ms	remaining: 1.14s
321:	learn: 0.3485921	total: 539ms	remaining: 1.13s
322:	learn: 0.3485610	total: 540ms	remaining: 1.13s
323:	learn: 

485:	learn: 0.3279557	total: 831ms	remaining: 878ms
486:	learn: 0.3277495	total: 832ms	remaining: 877ms
487:	learn: 0.3276446	total: 834ms	remaining: 875ms
488:	learn: 0.3275746	total: 836ms	remaining: 873ms
489:	learn: 0.3273956	total: 837ms	remaining: 872ms
490:	learn: 0.3272118	total: 839ms	remaining: 870ms
491:	learn: 0.3271582	total: 841ms	remaining: 868ms
492:	learn: 0.3271122	total: 843ms	remaining: 867ms
493:	learn: 0.3270275	total: 844ms	remaining: 865ms
494:	learn: 0.3267209	total: 846ms	remaining: 863ms
495:	learn: 0.3266401	total: 847ms	remaining: 861ms
496:	learn: 0.3265699	total: 849ms	remaining: 859ms
497:	learn: 0.3265342	total: 850ms	remaining: 857ms
498:	learn: 0.3265071	total: 852ms	remaining: 855ms
499:	learn: 0.3263948	total: 853ms	remaining: 853ms
500:	learn: 0.3262538	total: 855ms	remaining: 852ms
501:	learn: 0.3261581	total: 857ms	remaining: 851ms
502:	learn: 0.3260365	total: 859ms	remaining: 849ms
503:	learn: 0.3258939	total: 860ms	remaining: 847ms
504:	learn: 

674:	learn: 0.3083153	total: 1.15s	remaining: 555ms
675:	learn: 0.3081888	total: 1.15s	remaining: 553ms
676:	learn: 0.3080961	total: 1.16s	remaining: 552ms
677:	learn: 0.3079540	total: 1.16s	remaining: 550ms
678:	learn: 0.3079411	total: 1.16s	remaining: 548ms
679:	learn: 0.3078609	total: 1.16s	remaining: 546ms
680:	learn: 0.3077292	total: 1.16s	remaining: 545ms
681:	learn: 0.3076707	total: 1.17s	remaining: 543ms
682:	learn: 0.3076034	total: 1.17s	remaining: 542ms
683:	learn: 0.3074473	total: 1.17s	remaining: 540ms
684:	learn: 0.3073078	total: 1.17s	remaining: 539ms
685:	learn: 0.3072553	total: 1.17s	remaining: 537ms
686:	learn: 0.3071420	total: 1.18s	remaining: 536ms
687:	learn: 0.3070689	total: 1.18s	remaining: 534ms
688:	learn: 0.3069150	total: 1.18s	remaining: 533ms
689:	learn: 0.3068464	total: 1.18s	remaining: 531ms
690:	learn: 0.3067848	total: 1.19s	remaining: 530ms
691:	learn: 0.3066248	total: 1.19s	remaining: 528ms
692:	learn: 0.3066147	total: 1.19s	remaining: 526ms
693:	learn: 

854:	learn: 0.2894422	total: 1.49s	remaining: 253ms
855:	learn: 0.2894196	total: 1.5s	remaining: 252ms
856:	learn: 0.2893531	total: 1.5s	remaining: 250ms
857:	learn: 0.2891981	total: 1.5s	remaining: 248ms
858:	learn: 0.2890788	total: 1.5s	remaining: 246ms
859:	learn: 0.2890464	total: 1.5s	remaining: 245ms
860:	learn: 0.2890231	total: 1.5s	remaining: 243ms
861:	learn: 0.2887692	total: 1.51s	remaining: 241ms
862:	learn: 0.2886077	total: 1.51s	remaining: 240ms
863:	learn: 0.2885099	total: 1.51s	remaining: 238ms
864:	learn: 0.2884695	total: 1.51s	remaining: 236ms
865:	learn: 0.2883685	total: 1.51s	remaining: 234ms
866:	learn: 0.2882939	total: 1.52s	remaining: 233ms
867:	learn: 0.2882331	total: 1.52s	remaining: 231ms
868:	learn: 0.2880618	total: 1.52s	remaining: 229ms
869:	learn: 0.2877895	total: 1.52s	remaining: 228ms
870:	learn: 0.2877410	total: 1.52s	remaining: 226ms
871:	learn: 0.2876343	total: 1.53s	remaining: 224ms
872:	learn: 0.2875561	total: 1.53s	remaining: 222ms
873:	learn: 0.2874

In [217]:
results_ord

,ModelName,Accuracy,Bal Acc.,Time
0,Decision Tree,80.134680,78.753236,0.146607
1,Extra Trees,82.716049,80.462084,10.335347
2,Random Forest,81.481481,80.176877,1.679506
3,AdaBoost,81.144781,79.903653,0.910563
4,Skl GBM,83.277217,81.027706,1.020271
5,SKl HistGBM,82.491582,80.831176,8.794470
6,XGBoost,81.930415,80.430927,2.890266
7,LightGBM,82.491582,80.886300,1.369337
8,CatBoost,83.277217,81.193078,22.723006


In [218]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [229]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data

### BEGIN SOLUTION
best_model = tree_classifiers[results_ord.iloc[8].ModelName]
best_model.fit(x,y)
### END SOLUTION

Learning rate set to 0.009807
0:	learn: 0.6861571	total: 1.62ms	remaining: 1.62s
1:	learn: 0.6793269	total: 3.13ms	remaining: 1.56s
2:	learn: 0.6730029	total: 4.61ms	remaining: 1.53s
3:	learn: 0.6678537	total: 5.41ms	remaining: 1.35s
4:	learn: 0.6611947	total: 6.87ms	remaining: 1.37s
5:	learn: 0.6553076	total: 8.31ms	remaining: 1.38s
6:	learn: 0.6497487	total: 9.84ms	remaining: 1.4s
7:	learn: 0.6435490	total: 11.3ms	remaining: 1.4s
8:	learn: 0.6376790	total: 12.8ms	remaining: 1.41s
9:	learn: 0.6322433	total: 14.4ms	remaining: 1.42s
10:	learn: 0.6262971	total: 15.9ms	remaining: 1.43s
11:	learn: 0.6205698	total: 17.3ms	remaining: 1.42s
12:	learn: 0.6164513	total: 18.1ms	remaining: 1.37s
13:	learn: 0.6112425	total: 19.5ms	remaining: 1.38s
14:	learn: 0.6061658	total: 21ms	remaining: 1.38s
15:	learn: 0.6010704	total: 22.5ms	remaining: 1.38s
16:	learn: 0.5965741	total: 23.7ms	remaining: 1.37s
17:	learn: 0.5924102	total: 25.3ms	remaining: 1.38s
18:	learn: 0.5890321	total: 26.6ms	remaining: 1.

238:	learn: 0.3704615	total: 336ms	remaining: 1.07s
239:	learn: 0.3702406	total: 337ms	remaining: 1.07s
240:	learn: 0.3701975	total: 338ms	remaining: 1.06s
241:	learn: 0.3698057	total: 340ms	remaining: 1.06s
242:	learn: 0.3696096	total: 342ms	remaining: 1.06s
243:	learn: 0.3693526	total: 343ms	remaining: 1.06s
244:	learn: 0.3691163	total: 360ms	remaining: 1.11s
245:	learn: 0.3688328	total: 361ms	remaining: 1.11s
246:	learn: 0.3686134	total: 363ms	remaining: 1.1s
247:	learn: 0.3684018	total: 364ms	remaining: 1.1s
248:	learn: 0.3681903	total: 366ms	remaining: 1.1s
249:	learn: 0.3681089	total: 367ms	remaining: 1.1s
250:	learn: 0.3680228	total: 368ms	remaining: 1.1s
251:	learn: 0.3679355	total: 370ms	remaining: 1.1s
252:	learn: 0.3677887	total: 371ms	remaining: 1.09s
253:	learn: 0.3676070	total: 373ms	remaining: 1.09s
254:	learn: 0.3674068	total: 374ms	remaining: 1.09s
255:	learn: 0.3671327	total: 376ms	remaining: 1.09s
256:	learn: 0.3669459	total: 377ms	remaining: 1.09s
257:	learn: 0.3667

463:	learn: 0.3365596	total: 680ms	remaining: 786ms
464:	learn: 0.3363987	total: 682ms	remaining: 785ms
465:	learn: 0.3362185	total: 684ms	remaining: 784ms
466:	learn: 0.3359698	total: 686ms	remaining: 782ms
467:	learn: 0.3357762	total: 687ms	remaining: 781ms
468:	learn: 0.3357172	total: 688ms	remaining: 779ms
469:	learn: 0.3354781	total: 690ms	remaining: 778ms
470:	learn: 0.3353423	total: 692ms	remaining: 777ms
471:	learn: 0.3352995	total: 693ms	remaining: 776ms
472:	learn: 0.3351541	total: 695ms	remaining: 775ms
473:	learn: 0.3350586	total: 697ms	remaining: 773ms
474:	learn: 0.3349255	total: 698ms	remaining: 772ms
475:	learn: 0.3348592	total: 700ms	remaining: 771ms
476:	learn: 0.3345495	total: 702ms	remaining: 769ms
477:	learn: 0.3344519	total: 704ms	remaining: 768ms
478:	learn: 0.3342480	total: 706ms	remaining: 768ms
479:	learn: 0.3339928	total: 708ms	remaining: 767ms
480:	learn: 0.3338643	total: 709ms	remaining: 765ms
481:	learn: 0.3336220	total: 711ms	remaining: 764ms
482:	learn: 

668:	learn: 0.3137173	total: 1.02s	remaining: 506ms
669:	learn: 0.3136340	total: 1.02s	remaining: 504ms
670:	learn: 0.3135486	total: 1.02s	remaining: 503ms
671:	learn: 0.3131995	total: 1.03s	remaining: 501ms
672:	learn: 0.3131105	total: 1.03s	remaining: 500ms
673:	learn: 0.3129650	total: 1.03s	remaining: 499ms
674:	learn: 0.3127331	total: 1.03s	remaining: 497ms
675:	learn: 0.3126380	total: 1.03s	remaining: 496ms
676:	learn: 0.3125136	total: 1.04s	remaining: 495ms
677:	learn: 0.3123681	total: 1.04s	remaining: 493ms
678:	learn: 0.3122546	total: 1.04s	remaining: 492ms
679:	learn: 0.3121765	total: 1.04s	remaining: 491ms
680:	learn: 0.3120875	total: 1.04s	remaining: 489ms
681:	learn: 0.3119809	total: 1.04s	remaining: 488ms
682:	learn: 0.3119306	total: 1.05s	remaining: 486ms
683:	learn: 0.3118790	total: 1.05s	remaining: 485ms
684:	learn: 0.3117906	total: 1.05s	remaining: 483ms
685:	learn: 0.3117473	total: 1.05s	remaining: 482ms
686:	learn: 0.3116450	total: 1.05s	remaining: 480ms
687:	learn: 

885:	learn: 0.2911138	total: 1.36s	remaining: 176ms
886:	learn: 0.2910014	total: 1.37s	remaining: 174ms
887:	learn: 0.2908295	total: 1.37s	remaining: 173ms
888:	learn: 0.2907314	total: 1.37s	remaining: 171ms
889:	learn: 0.2905602	total: 1.37s	remaining: 170ms
890:	learn: 0.2903891	total: 1.37s	remaining: 168ms
891:	learn: 0.2902679	total: 1.37s	remaining: 166ms
892:	learn: 0.2901530	total: 1.38s	remaining: 165ms
893:	learn: 0.2900759	total: 1.38s	remaining: 163ms
894:	learn: 0.2900041	total: 1.38s	remaining: 162ms
895:	learn: 0.2899535	total: 1.38s	remaining: 160ms
896:	learn: 0.2899050	total: 1.38s	remaining: 159ms
897:	learn: 0.2898187	total: 1.39s	remaining: 157ms
898:	learn: 0.2897122	total: 1.39s	remaining: 156ms
899:	learn: 0.2896293	total: 1.39s	remaining: 154ms
900:	learn: 0.2895208	total: 1.39s	remaining: 153ms
901:	learn: 0.2894060	total: 1.39s	remaining: 151ms
902:	learn: 0.2893623	total: 1.39s	remaining: 150ms
903:	learn: 0.2893064	total: 1.4s	remaining: 148ms
904:	learn: 0

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=10))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000001DB94BB4E48>)])

In [228]:
results_ord.iloc[8]

ModelName     CatBoost
Accuracy     83.277217
Bal Acc.     81.193078
Time         22.723006
Name: 8, dtype: object

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [230]:
# test_pred = # Get the predictions for x_test

### BEGIN SOLUTION
test_pred = best_model.predict(x_test)
### END SOLUTION

In [231]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [51]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [52]:
sub.to_csv("sub.csv")

In [53]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
